##### Тестирование списка договоров, которые будут учитываться при расчете фичей 

сравнение  "data/external/portfolio_tsal_for_test.pkl" - портфеля ЦАЛ (глубиной 5 лет) с "data/interm/test/lst_contracts.pkl" - теми договорами, которые будут учитываться на дату принятия решения

In [1]:
pre = '../../' # путь до главной директории проекта
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init.ipynb #подгрузить нужные библиотеки

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/test_portfolio.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/test/lst_contracts.pkl"
lst_contracts = pd.read_pickle(name, compression={'method':'zip'})

print(len(lst_contracts))

# lst_contracts.head(1)

191677


In [4]:
name = pre + "data/external/portfolio_tsal_for_test.pkl"
portfolio_tsal = pd.read_pickle(name, compression={'method':'zip'})

portfolio_tsal['CONTRACT_KEY'] = portfolio_tsal['CONTRACT_KEY'].astype(str) 

print(len(portfolio_tsal))
# portfolio_tsal.head(1)

203571


In [ ]:
num, duplicates = identify_duplicates(portfolio_tsal, 'CONTRACT_KEY')
Check_dupl(num, duplicates)

In [6]:
portfolio_tsal = portfolio_tsal.drop_duplicates(subset='CONTRACT_KEY')

In [7]:
inn_ctr_cnt_tsal = portfolio_tsal.groupby(['NEW_INN'])['CONTRACT_NUM'].nunique().reset_index()
# inn_ctr_cnt_tsal.head(1)

In [8]:
df_cnt = lst_contracts.groupby(['ИНН контрагента'])['CONTRACT_NUM'].nunique().to_frame().reset_index()

# df_cnt.head(1)

In [9]:
%run ../init.ipynb #подгрузить нужные библиотеки

kwargs = dict( sourcesName = ['test', 'tsal'], 
               suffix = '_tsal',
               left_on = 'ИНН контрагента', right_on = 'NEW_INN'
             )

comp =  ComparisonDataFrames(df_cnt, inn_ctr_cnt_tsal, **kwargs) 
df = comp.presentRows()  
# df.head()

одновременно присутствуют в test и tsal 67551 строк


In [10]:
df = comp.check_fields('CONTRACT_NUM', 'CONTRACT_NUM')
# df.head(1)

Проверка по столбцам CONTRACT_NUM и CONTRACT_NUM_tsal
значения не совпадают у 12 строк из 67551


In [ ]:
df.sort_values(by='ИНН контрагента')

По ним не было передачи в лизинг. В ЦАЛ сделали не правильно отбор на дату лизинга

### Заключение 
Проверка успешно прошла
значения не совпадают у 12 строк из 67551
связано с тем, что Отбор строк по дате лизинга в ЦАЛ производится после того как дата лизинга агрегирована на самое первое приложение, получается, что если у первого приложения не было передачи в лизинг, а у повторного есть, то невозможно определить что есть лизинг. Это необходимо править в витрине ЦАЛ

In [14]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 